<a href="https://colab.research.google.com/github/seunghee0518/R_File/blob/main/R_%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95_5%EC%9E%A5_%EC%9E%90%EB%A3%8C%EC%9D%98%EC%A7%84%EB%8B%A8(%ED%8A%B9%EC%9D%B4%EC%A0%90%26%EC%98%81%ED%96%A5%EB%A0%A5%EC%B8%A1%EC%A0%95).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 특이점 검정 : Bonferroni t검정
* <표 5.1 Forbes 자료>
* num : 번호
* temp : 물끊는 온도
* press : 대기압력

In [ ]:
#데이터 불러오기
forbes.url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95/forbes.txt'
forbes = read.table(forbes.url, header=T)
head(forbes, 3)
attach(forbes)

In [ ]:
#Lpress : 대기압력에 상용로그 취함 & 100곱함
forbes$Lpress = 100*log10(forbes$press)
head(forbes,3)

In [ ]:
#산점도
plot(forbes$temp, forbes$Lpress, pch = 19)

#케이스 번호 확인
print(identify(forbes$temp, forbes$Lpress))

In [ ]:
#회귀분석
forbes.lm = lm(Lpress ~ temp, data = forbes)
summary(forbes.lm)

In [ ]:
#분산분석표
anova(forbes.lm)

In [ ]:
#잔차분석
forbes.res = ls.diag(forbes.lm)
names(forbes.res)

In [ ]:
#표준화 잔차, 스튜던트화 잔차, Hii값 출력
resid.result = cbind(forbes.res$std.res, forbes.res$stud.res, forbes.res$hat)
colnames(resid.result) = c('standardized resid', 'studentized resid', 'Hat')
resid.result = round(resid.result, 3)
print(resid.result)

* Hii값은 0과 1사이의 값으로 중심에서 멀리 떨어져 있을수록 큰 값을 가짐
* 12번째 스튜던트화 잔차가 가장 큼
* 통계량의 자유도(n-k-2) : 17-1-2 = 14

In [ ]:
#12번째 관측값 특이점 검정 -> Bonferroni보정 p값
2*17*(1-pt(12.374, 14))

* 유의확률 p값이 매우 작으므로 특이값으로 판단됨

In [ ]:
#유의수준 0,01의 기각치 계산
qt(0.01/(2*17), 14)

In [ ]:
#패키지 car - outlierTest()함수 이용
install.packages('car')
library(car)
outlierTest(forbes.lm)

In [ ]:
#비교 : rstudent()
rstudent(forbes.lm)

#관측값의 영향력 측정
* <표 5.3> Prater의 가솔린 자료
* X1 : 원유중력
* X2 : 원유증기압력
* X3 : 원유 ASTM(10%원유가 증발했을때 온도)
* X4 : 가솔린 증발온도(모든 가솔린 증발)
* 일반적으로 가벼운 원유가 무거운 원유보다 더 많은 가솔린을 만듦
* 일정한 양의 원유가 주어졌을 때는 증발하는 온도가 높아질수록 많은 양의 가솔린이 추출됨

In [ ]:
#데이터 가져오기
prater.url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95/prater.txt'
prater = read.table(prater.url, header = T)
head(prater, 2)

In [ ]:
#회귀모형 적합
prater.lm = lm(Y ~ X1+X2+X3+X4, data = prater)
summary(prater.lm)

In [ ]:
#단계별 선택방법으로 모형적합
start.lm = lm(Y ~ 1, data = prater)
full.lm = prater.lm
step(start.lm, scope = list(upper = full.lm), data = prater, direction = 'both')

* 최종적으로 선택된 변수 : X3, X4

In [ ]:
#^B3(i), ^B4(i) 계산
hat = vector()
for(i in 1:32){
  hat.lm = lm(Y ~ X3+X4, data = prater[-i, ])
  except = hat.lm$coefficients[-1]
  hat = rbind(hat, except)
}

number = c(1:32)
rownames(hat) = number
head(hat, 3)
tail(hat, 3)

In [ ]:
#hat B3, B4의 산점도
plot(hat[, 1], hat[, 2], xlab = 'hat_B3(i)', ylab = 'hat_B4(i)', pch = 19)
text(hat[, 1], hat[, 2], labels = rownames(hat), pos = 1)

* 11, 32번 관측치가 다른 추정값으로부터 상대적으로 멀리 떨어져 있음
* 추정값의 변화가 얼마나 큰지 판단하는 기준 : cook의 통계량 Di

# cook의 Di통계량으로 영향력 측정
* <표 5.5 >토양침식 자료
* SL : 토양의 침식량(1에이커당 파운드)
* SG : 땅의 경사도
* LOBS : 토양침식이 가장 심한 곳의 깊이(인치)
* PGC : 흙으로 덮여진 부분의 비율

In [ ]:
soil_url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95/soil.txt'
soil = read.table(soil_url, header = T)
head(soil, 3)

In [ ]:
#회귀모형 적합
soil.lm = lm(SL ~ SG+LOBS+PGC, data = soil)
summary(soil.lm)

In [ ]:
anova(soil.lm)

In [ ]:
#(잔차, 추정값) 산점도
plot(soil.lm$fitted, soil.lm$resid, pch = 19)
text(soil.lm$fitted, soil.lm$resid, labels = soil$obs, pos = 1)

* 7번 관측값 : 예측값이 가장 크므로 영향력 있는 관측값임
* 10번 관측값 : 7번보다는 중간에 가깝고, 잔차가 크므로 특이점이 될 가능성이 높아보임

In [ ]:
#cook의 Di통계량
soil.diag = ls.diag(soil.lm)
names(soil.diag)

In [ ]:
diag.st = cbind(soil$obs,soil.diag$hat, soil.diag$std.res, soil.diag$stud.res, soil.diag$cooks)
colnames(diag.st) = c('num', 'Hii', 'ri', 'ti', 'Di')
round(diag.st, 3)

In [ ]:
Di = cooks.distance(soil.lm)
round(Di, 3)

* cook거리에서 7번 관찰값이 1.227로 영향력 있는 관찰값으로 판명됨

In [ ]:
library(car)
outlierTest(soil.lm)

* car패키지를 이용한 outlierTest결과에서 10번 관찰값의 Bonferonni p가 0.093으로 특이점으로는 판명되지 않음을 알 수 있음